# uep239 Final project: Where the Book Worms Go
## By Kees Schipper

### Start by loading in all libraries in one cell. 
This cell is going to be adjusted while writing the notebook based on what packages I might need

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

import rasterio
from rasterio.plot import show
from rasterio import features

import richdem as rd
from scipy import ndimage
from shapely.geometry import box
from rasterstats import zonal_stats
import os

# finally, check working directory to see where we are in our file structure
os.getcwd()

'C:\\Users\\keess\\OneDrive\\Documents\\Tufts Friedman School\\Semester 4\\Geospatial Programming with Python\\uep239FinalProj\\Data'

### Change current working directory to the data folder for easier reading of files

In [4]:
os.chdir("../Data")

### Start reading in data files

In [45]:
# tabular data
AgeSex = pd.read_csv("AgeACS.csv", header = 0, skiprows=[1])
Edu = pd.read_csv("EduAttainmentACS.csv", header = 0, skiprows = [1])

# select only columns that have data of interest to you, plus columns containing primary keys
AgeSex_cols = AgeSex[['GEO_ID', 'NAME', 'S0101_C01_022E', 'S0101_C01_026E']].copy()
Edu_cols = Edu[['GEO_ID', 'NAME', 'S1401_C03_010E', 'S1401_C04_010E']].copy()

# rename columns to be more intuitive
AgeSex_cols = AgeSex_cols.rename(columns = {"S0101_C01_022E":"popUnder18", "S0101_C01_026E":"popOver18"})
Edu_cols = Edu_cols.rename(columns = {"S1401_C03_010E":"popInCollege", "S1401_C04_010E":"pctInCollege"})

print(AgeSex_cols.columns)
print(Edu_cols.columns)

ValueError: could not convert string to float: '-'

#### Let's also look at some basic descriptions of our data to see what we're dealing with

In [40]:
AgeSex_cols.describe()

,popUnder18,popOver18
count,537.000000,537.000000
mean,2553.556797,10203.417132
std,2702.400435,10291.627226
min,0.000000,0.000000
25%,404.000000,2141.000000
50%,1656.000000,6988.000000
75%,3771.000000,14533.000000
max,16734.000000,51217.000000


In [46]:
Edu_cols.describe()

,popInCollege
count,537.000000
mean,519.957169
std,859.161892
min,0.000000
25%,74.000000
50%,282.000000
75%,600.000000
max,10248.000000


Interesting...it seems like pctInCollege is not a numeric object. Let's explore why that is

In [49]:
print(Edu_cols.dtypes) # pctInCollege is an object, likely meaning there is some type of character string preventing it from being numeric
print(Edu_cols.pctInCollege.unique())

GEO_ID          object
NAME            object
popInCollege     int64
pctInCollege    object
dtype: object
['48.8' '75.7' '90.7' '72.1' '62.8' '37.8' '0.0' '40.3' '75.3' '100.0'
 '60.4' '72.3' '72.4' '65.5' '70.5' '66.1' '-' '56.1' '30.6' '60.0' '64.9'
 '74.7' '77.2' '61.3' '78.3' '77.0' '81.1' '87.8' '62.3' '66.5' '41.4'
 '35.4' '56.0' '64.3' '48.5' '82.4' '64.4' '36.8' '73.4' '87.3' '90.1'
 '78.1' '75.5' '81.3' '90.4' '61.5' '72.8' '40.0' '61.1' '57.6' '50.0'
 '81.6' '85.7' '24.6' '71.9' '79.8' '63.9' '44.4' '91.0' '63.8' '83.6'
 '63.3' '62.7' '93.4' '63.7' '21.5' '77.1' '58.8' '35.8' '72.2' '27.8'
 '76.2' '56.3' '65.4' '37.0' '57.1' '19.6' '55.6' '24.2' '82.8' '68.8'
 '86.7' '91.1' '46.9' '66.7' '47.2' '84.2' '87.1' '47.7' '85.0' '85.2'
 '63.2' '46.0' '72.7' '68.4' '76.6' '97.7' '50.8' '71.8' '57.9' '47.9'
 '43.5' '84.5' '58.6' '70.7' '67.3' '71.0' '78.7' '67.4' '47.3' '84.3'
 '81.0' '91.8' '90.2' '67.2' '65.1' '34.9' '78.5' '61.6' '69.4' '83.5'
 '47.6' '54.6' '88.4' '54.9' '49.9' '7

In the unique values, we see that there is one entry that is a `'-'` instead of a number, which is preventing python from interpreting this series as a float. Let's remove that string value and change it to 0.0, then we can convert the series to a float.

In [53]:
# replace the problematic string
Edu_cols.pctInCollege = Edu_cols.pctInCollege.str.replace(pat = '-', repl = '0.0', regex = False)

# change the series into a float data type
Edu_cols.pctInCollege = Edu_cols.pctInCollege.astype("float")


AttributeError: Can only use .str accessor with string values!

In [56]:
# check the dtype and unique values of the series
print(Edu_cols.pctInCollege.dtype)
Edu_cols.describe()

float64


,popInCollege,pctInCollege
count,537.000000,537.000000
mean,519.957169,55.576536
std,859.161892,25.895560
min,0.000000,0.000000
25%,74.000000,40.700000
50%,282.000000,60.300000
75%,600.000000,74.400000
max,10248.000000,100.000000
